In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# **Question 1**

In [ ]:
#Loading of the numpy file and the images
img1 = cv.imread("img1.jpg",0)
img2 = cv.imread("img2.jpg",0)
I = [img1,img2]
a = np.load("GT2.npy")


In [ ]:
# function to calculate the partial derivatives
# values t can take are restricted to 0 and 1

def Ix(I,k,l,t):
  I = np.array(I).astype('float64')
  temp1 = I[t][k+1][l] + I[t][k+1][l+1] + I[t+1][k+1][l] + I[t+1][k+1][l+1]
  temp2 = I[t][k][l] + I[t][k][l+1] + I[t+1][k][l] + I[t+1][k][l+1]

  return 0.25*(temp1 - temp2)

def Iy(I,k,l,t):
  I = np.array(I).astype('float64')
  temp1 = I[t][k][l+1] + I[t][k+1][l+1] + I[t+1][k][l+1] + I[t+1][k+1][l+1]
  temp2 = I[t][k][l] + I[t][k+1][l] + I[t+1][k][l] + I[t+1][k+1][l]

  return 0.25*(temp1 - temp2)

def It(I,k,l,t):
  I = np.array(I).astype('float64')
  temp1 = I[t+1][k][l] + I[t+1][k][l+1] + I[t+1][k+1][l] + I[t+1][k+1][l+1]
  temp2 = I[t][k][l] + I[t][k][l+1] + I[t][k+1][l] + I[t][k+1][l+1]

  return 0.25*(temp1 - temp2)
  

In [ ]:
# padding of the image
def padImage(img,size):
  w = size//2
  img = np.pad(img,((w,w),(w,w)), 'constant', constant_values = 0)
  return img

In [ ]:
# Calculation of optical flow at a particular pixel location using window size s

def opflow(I,k,l,size):
  w = size//2
  t = 0
  A = []
  B = []
  for i in range(k-w,k+1+w):
    for j in range(l-w,l+1+w):
      A.append([Ix(I,i,j,t),Iy(I,i,j,t)])
      B.append(It(I,i,j,t))

  u = np.linalg.lstsq(A,B,)
  u = u[0]
  temp = np.sqrt(u[0]**2 + u[1]**2)

  return temp



In [ ]:
## Calculating the optical flow at every pixel

def compute_opflow(I,size):
  w = size//2
  u = []
  for i in range(w,len(I[0])-2*w):
    print("In the ",i,"th row")
    r = []
    for j in range(w,len(I[0][0])-2*w):
      t = opflow(I,i,j,size)
      r.append(t)
    u.append(r)

  return u



# **Question 2**

**Exhaustive** **Search**

In [ ]:
## function to calulate the cost function
## I[0] = reference frame
## I[1] = current frame

def MSE(k,l,I):
  s = 0
  I = np.array(I).astype('float64')
  for i in range(k,k+16):
    for j in range(l,l+16):
      s += (I[1][i][j] - I[0][i][j]) ** 2
  return s/256


In [ ]:
## function to find the matching block
## p is the search parameter defining the search window

def blockMatch(k,l,I,p):
  I = np.array(I).astype('float64')
  min = MSE(k-p,l-p,I)
  xloc,yloc = k-p,l-p
  for i in range(-p,p):
    for j in range(-p,p):
      if min < MSE(k+i,l+j,I):
        min = MSE(k+i,l+j,I)
        xloc = k+i
        yloc = l+j
  return xloc,yloc

In [ ]:
# Performing the exhaustive search
# I is a padded image

def PerformExhaustiveSearch(I,p):
  I = np.array(I).astype('float64')
  vf = len(I[0])-16-p
  hf = len(I[0][0])-16-p
  print(hf)
  u = np.zeros((len(I[0]),len(I[0][0])))
  for i in range(p,vf,16):
    print("In the ",i,"th row")
    for j in range(p,hf,16):
      print("In the ",j,"th col")
      xloc, yloc = blockMatch(i,j,I,p)
      print('xloc,yloc = ',xloc,yloc)
      for m in range(16):
        for n in range(16):
          u[i+m][j+n] = I[1][i+m][j+n]-I[0][m+xloc][n+yloc]

  return u

In [ ]:
imgPad = [padImage(I[0],14),padImage(I[1],14)]